### CALCULATE FEATURES BASED ON FOOD INSPECTION DATA

In [142]:
import numpy as np
import pandas as pd
import os.path

root_path = os.path.dirname(os.getcwd())

# Load food inspection data
inspections = pd.read_csv(os.path.join(root_path, "DATA/food_inspections.csv"))

# Create basis for model_data (risk not included in Chicago repository)
data = inspections[["inspection_date", "license", "inspection_id", "facility_type", "results", "risk"]]

In [143]:
# Load violation data
values = pd.read_csv(os.path.join(root_path, "DATA/violation_values.csv"))
counts = pd.read_csv(os.path.join(root_path, "DATA/violation_counts.csv"))

# Merge with violation data
data = pd.merge(data, values, on="inspection_id")
data = pd.merge(data, counts, on="inspection_id")

In [144]:
# Create pass / fail flags
data["pass_flag"] = data.results.map(lambda x: 1 if x == "Pass" else 0)
data["fail_flag"] = data.results.map(lambda x: 1 if x == "Fail" else 0)

In [145]:
# Sort inspections by date
grouped = data.sort_values(by="inspection_date", inplace=True)

# Find previous inspections by shifting each license group
past_data = data.groupby("license").shift(1)

In [146]:
# Add past fails
data["past_fail"] = past_data.fail_flag

# Add past violation counts
data["past_critical"] = past_data.critical_count
data["past_serious"] = past_data.serious_count
data["past_minor"] = past_data.minor_count

In [147]:
# Select past violation values, remove inspection_id
past_values = past_data[values.columns].drop("inspection_id", axis=1).add_prefix("p")

# Set violation values to 0 for first inspections
past_values.fillna(0, inplace=True)

# Add past values to model data
data = data.join(past_values)

In [148]:
# Calculate time since previous inspection
deltas = pd.to_datetime(data.inspection_date) - pd.to_datetime(past_data.inspection_date)

# Add years since previous inspection
data["time_since_last"] = deltas.apply(lambda x: x.days / 365.25)

In [149]:
# Check if first record
data["first_record"] = data.time_since_last.map(lambda x: 1 if pd.isnull(x) else 0)

# Set time since last for first inspections to 2
data.time_since_last.fillna(2, inplace=True)

In [9]:
# REMEMBER TO INCLUDE PRIORITY FEATURE

In [154]:
len(data)

56392

In [178]:
len(data)

56392

In [174]:
len(thingy)

56392

In [167]:
len(data) + len(age_data)

112034

### CALCULATE FEATURES BASED ON BUSINESS LICENSE DATA

In [153]:
# Load business license data
licenses = pd.read_csv(os.path.join(root_path, "DATA/business_licenses.csv"))

/home/russell/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,6,23,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [156]:
# Match by license number and when inspection_date
# falls wrt business license renewal cycle
def get_business_id(inspection):
    matches = licenses.loc[licenses.license_number == inspection.license]
    matches = matches.loc[
        (matches.license_start_date <= inspection.inspection_date) &
        (matches.expiration_date > inspection.inspection_date)
    ]
    if len(matches) > 0:
        return matches.iloc[0].id
    return np.nan

# Sort licenses to allow selection of first match
licenses.sort_values("license_start_date", inplace=True)

# 3 mins
data["business_id"] = data.apply(get_business_id, axis=1)

In [158]:
# Merge with business licenses to create temporary dataframe
temp = pd.merge(data, licenses, left_on="business_id", right_on="id")

# Convert dates to datetime
temp.inspection_date = pd.to_datetime(temp.inspection_date)
temp.license_start_date = pd.to_datetime(temp.license_start_date)

In [170]:
def get_age_data(group):
    min_date = group.license_start_date.min()
    deltas = group.inspection_date - min_date
    group["age_at_inspection"] = deltas.apply(lambda x: x.days / 365.25)
    return group[["inspection_id", "age_at_inspection"]]

# Group by licenses and get age at inspection
age_data = temp.groupby("license").apply(get_age_data)

In [176]:
# Merge in age_at_inspection
data = pd.merge(data, age_data, on="inspection_id", how="left")

### ATTACH KDE DATA

### ATTACH WEATHER DATA

In [11]:
# Load weather data
weather = pd.read_csv(os.path.join(root_path, "DATA/weather.csv"))

# Merge weather data with model data
data = pd.merge(data, weather, on="inspection_id")